# Cutout Region

In [1]:
%run helpers/example_setup.ipynb

id: 1

clean design created with default db, cell, layout, gnd_net variables.


## Setup nets

In [2]:
# Nets
net_names = ['Signal_1', 'Signal_2', 'Signal_3']
for name in net_names:
    Net.create(layout, name)

## Setup Layers

In [3]:
"""Create layers, add them to layer collection and retrieve them again."""

# Layers in layout's layer collection
insulator_layer_1 = StackupLayer.create('D1', LayerType.DIELECTRIC_LAYER, 0.0002, 0.0002, "FR4_epoxy")
layer_1 = StackupLayer.create('S1', LayerType.SIGNAL_LAYER, 0.0001, 0.0003, "copper")
insulator_layer_2 = StackupLayer.create('D2', LayerType.DIELECTRIC_LAYER, 0.0001, 0.0004, "FR4_epoxy")
layer_2 = StackupLayer.create('S2', LayerType.SIGNAL_LAYER, 0.0001, 0.0005, "copper")
insulator_layer_3 = StackupLayer.create('D3', LayerType.DIELECTRIC_LAYER, 0.0003, 0.0005, "FR4_epoxy")
layer_3 = StackupLayer.create('S3', LayerType.SIGNAL_LAYER, 0.0001, 0.0007, "copper")

# Layers for additional cells
lc = layout.layer_collection
lc.mode = LayerCollectionMode.OVERLAPPING
lc.add_layers([insulator_layer_1, layer_1, insulator_layer_2, layer_2, insulator_layer_3, layer_3])

# Retrieve layers
layer_1 = lc.find_by_name('S1')
layer_2 = lc.find_by_name('S2')
layer_3 = lc.find_by_name('S3')

# Print layers created for lc
print("LC", [(l.name) for l in lc.get_layers() ])

LC ['S3', 'D3', 'S2', 'D2', 'S1', 'D1']


## Create geometry

In [4]:
"""Create Primitive objects"""
# lambda for value resolution
um = lambda val : val * 1e-6

# create rectangles in different layers
rect = Rectangle.create(layout=layout,
                        layer=layer_1,
                        net='Signal_1',
                        rep_type=RectangleRepresentationType.LOWER_LEFT_UPPER_RIGHT,
                        param1=um(-46),
                        param2=um(9),
                        param3=um(-17),
                        param4=um(-14),
                        corner_rad=0.0,
                        rotation=0.0
                        )

rect_2 = Rectangle.create(layout=layout,
                          layer=layer_2,
                          net='Signal_2',
                          rep_type=RectangleRepresentationType.LOWER_LEFT_UPPER_RIGHT,
                          param1=um(-33),
                          param2=um(14),
                          param3=um(0),
                          param4=um(-3),
                          corner_rad=0.0,
                          rotation=0.0
                          )

rect_3 = Rectangle.create(layout=layout,
                          layer=layer_3,
                          net='Signal_3',
                          rep_type=RectangleRepresentationType.LOWER_LEFT_UPPER_RIGHT,
                          param1=um(-8),
                          param2=um(-11),
                          param3=um(-36),
                          param4=um(2),
                          corner_rad=0.0,
                          rotation=0.0
                          )

rect_4 = Rectangle.create(layout=layout,
                          layer=layer_1,
                          net='GND',
                          rep_type=RectangleRepresentationType.LOWER_LEFT_UPPER_RIGHT,
                          param1=um(-116),
                          param2=um(-11),
                          param3=um(-101),
                          param4=um(-20),
                          corner_rad=0.0,
                          rotation=0.0
                          )

#Retrieve the primitives
primitives_created = layout.primitives

# Print primitives created and their layers
print([(p.primitive_type.name, p.layer.name) for p in primitives_created ])

[('RECTANGLE', 'S1'), ('RECTANGLE', 'S2'), ('RECTANGLE', 'S3'), ('RECTANGLE', 'S1')]


## Create the PolygonData

In [5]:
# Redefine a lambda for point(x,y) resolution
um = lambda val_1, val_2 : (val_1 * 1e-6, val_2 * 1e-6)
cutout_region = PolygonData(
    [um(-7, 7), um(-23, -13), um(-39, -5), um(-38, 12 ), um(-20, 17)], closed=True
)

## Apply cutout region to cell

In [6]:
# Get nets
nets = layout.nets

# Cutout region
new_cutout_cell = cell.cutout(nets, nets, cutout_region)

#Retrieve the primitives from the new cell with the cutout region applied
primitives_of_cutout_region = new_cutout_cell.layout.primitives

# Print primitives created and their layers
print([(p.primitive_type.name, p.layer.name) for p in primitives_of_cutout_region ])

[('POLYGON', 'S1'), ('POLYGON', 'S2'), ('POLYGON', 'S3')]


In [7]:
%run helpers/example_teardown.ipynb